In [1]:
import os
import sys
import json

try:
    import pdfminer
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'pdfminer'])
finally:
    import pdfminer

try:
    import sklearn as sk
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'sklearn'])
finally:
    import sklearn as sk
    
try:
    import gensim as gn
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'gensim'])
finally:
    import gensim as gn
    
try:
    import texthero as hero
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'texthero'])
finally:
    import texthero as hero

try:
    import spacy
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install",'-U', 'spacy'])
finally:
    import spacy

try:
    import pandas as pd
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'pandas'])
finally:
    import pandas as pd


try:
    nlp = spacy.load("en_core_web_sm")
except ImportError:
    subprocess.check_call([sys.executable, "python", "-m", "spacy", 'download','en_core_web_sm'])
finally:
    nlp = spacy.load("en_core_web_sm")


try:
    import nltk
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", 'install','nltk'])
finally:
    import nltk



#for reading the pdf
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

#for identifying and removing common english language words
from nltk.corpus import stopwords

#for identifying and printing 3 word and 4 word combinations with their frequency of occurence
from nltk.collocations import TrigramCollocationFinder
from nltk.collocations import QuadgramCollocationFinder


#for counting the sentences and words

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
import collections
from nltk import word_tokenize
from collections import Counter


#for couting most frequent words
import re

def convert(filename, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(filename, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

pdfFiles = []
text_all = dict()
actual_words_list = list()
dir_name='./Source_Data/'
#print('list directory', os.listdir(dir_name))

for filename in os.listdir(dir_name):
    #print('filename is ', filename)
    if filename.endswith('.pdf') or filename.endswith('.PDF') or filename.endswith('.Pdf') or filename.endswith('.pDf') or filename.endswith('.pdF') or filename.endswith('.pDF') or filename.endswith('.pDf') or filename.endswith('.PDF'):
         pdfFiles.append(filename)
         text=convert(dir_name+filename)
         k=filename
         val=text
         dict([(k, []) for k in text_all])

         text_all[k] = val
         text_all
         
         sentence_count = len(nltk.tokenize.sent_tokenize(text))
         actual_words = nltk.tokenize.word_tokenize(text)
         actual_words_list.append(actual_words)
         
         word_count = len(nltk.tokenize.word_tokenize(text))
         print('\nThe file ',filename,' has ',word_count, 'words and ', sentence_count,' sentences in it.\n')
         
         #use findall for counting most common words, quadgrams, trigrams
         all_text = re.findall(r'\w+', text)
         s=pd.Series(text)
         all_text =map(lambda x: x.lower(), all_text)
         filtered_words = list(filter(lambda word: word not in stopwords.words('english') and word.isalpha(), all_text))
         #filtered_words = list(filter(lambda word: word not in stopwords.words('english'), all_text))

         word_counts = Counter(filtered_words).most_common(20)
         print('The 20 most commonly occuring words in this file are : \n\n', word_counts)
         
         print('\nThe 10 most common 3 word combinations appearing in this file are: \n')
         trigram = TrigramCollocationFinder.from_words(filtered_words)
         print(sorted(trigram.ngram_fd.items(), key=lambda t: (-t[1], t[0]))[:10])
         
         fourgrams=QuadgramCollocationFinder.from_words(filtered_words)
         print('\nThe 10 most common 4 word combinations appearing in this file are: \n')
         print(sorted(fourgrams.ngram_fd.items(), key=lambda t: (-t[1], t[0]))[:10])
         
         nlp = spacy.load("en_core_web_sm")
         doc = nlp(text)
         
         f = open("./Code_Output/spacy.txt", "a", encoding='utf-8')
         
         ent_dict = {}
   
         for ent in doc.ents:
            ent_dict.setdefault(ent.label_,[]).append(ent.text)
    
            for k,v in ent_dict.items():
                ent_dict[k]=list(set(v))
        
         print('\n Filename is:',filename, json.dumps(dict(ent_dict),indent=4), file=f)
         #print('\n Filename is:',filename, json.dumps(dict(ent_dict),indent=4))

            
         f1 = open("./Code_Output/texthero.txt", "a", encoding='utf-8')
         
         print('\nFilename is ',filename, file=f1)
         print('\n TFIDF Values are: ', (hero.tfidf(pd.Series(filtered_words), return_feature_names=True)), file=f1)
         print('\n Term Frequency Values are: ',hero.term_frequency(pd.Series(filtered_words), return_feature_names=True), file=f1)
         print('\n Top Words not-normalized are: ',hero.top_words(pd.Series(filtered_words), normalize=False), file=f1)
         print('\n TFIDF Words normalized are: ',hero.top_words(pd.Series(filtered_words), normalize=True), file=f1)
         
         
         print('----------------------------------------------------------------------------------------------------')


from sklearn.feature_extraction.text import TfidfVectorizer
actual_words_list=sum(actual_words_list,[])
myvocabulary = list(set(actual_words_list)) #['agreement', 'risk']
corpus = text_all #{1: "The game of life is a game of everlasting learning", 2: "The unexamined life is not worth living", 3: "Never stop learning"}
tfidf = TfidfVectorizer(vocabulary = myvocabulary, ngram_range = (1,2),stop_words='english')
tfs = tfidf.fit_transform(corpus.values())
df = pd.DataFrame(tfs.todense(), index=corpus.keys(), columns=myvocabulary) #, index=corpus_index, columns=feature_names)
df=df.transpose()
df=df.loc[~(df==0).all(axis=1)]
df.to_csv('./Code_Output/tfidfoutput.csv')
#print(df, file=f2)
print(df)


The file  Amendment Contract.pdf  has  289 words and  15  sentences in it.


The 10 most common 3 word combinations appearing in this file are: 

----------------------------------------------------------------------------------------------------

The file  Board Resolution - Approval of Option Grant Contract.pdf  has  350 words and  8  sentences in it.


The 10 most common 3 word combinations appearing in this file are: 

----------------------------------------------------------------------------------------------------

The file  docA.pdf  has  3263 words and  128  sentences in it.


The 10 most common 3 word combinations appearing in this file are: 

----------------------------------------------------------------------------------------------------

The file  ijfs-06-00007.pdf  has  8958 words and  515  sentences in it.


The 10 most common 3 word combinations appearing in this file are: 

-------------------------------------------------------------------------------------------

NameError: name 'f2' is not defined

In [103]:
tfidf = TfidfVectorizer(vocabulary = myvocabulary, ngram_range = (1,2),stop_words='english')
tfs = tfidf.fit_transform(corpus.values())
df = pd.DataFrame(tfs.todense(), index=corpus.keys(), columns=myvocabulary) #, index=corpus_index, columns=feature_names)
df=df.transpose()
df=df.loc[~(df==0).all(axis=1)]
df
#import seaborn as sns
#sns.heatmap(df)

,Amendment Contract.pdf,Board Resolution - Approval of Option Grant Contract.pdf,docA.pdf,ijfs-06-00007.pdf,Software License Agreement Contract.pdf
written,0.0,0.000000,0.054627,0.000000,0.021923
delle,0.0,0.000000,0.000000,0.010126,0.000000
concept,0.0,0.000000,0.000000,0.035439,0.000000
award,0.0,0.000000,0.046823,0.000000,0.016442
2012,0.0,0.000000,0.000000,0.025314,0.000000
...,...,...,...,...,...
exclusive,0.0,0.000000,0.000000,0.000000,0.054345
defects,0.0,0.000000,0.000000,0.000000,0.013586
lack,0.0,0.045091,0.000000,0.004085,0.000000
carried,0.0,0.000000,0.000000,0.020423,0.005481
